# Agregated metrics plot

Coco Yu

### Set Up Environment

Set up parameters as variables

In [ ]:
(study <- params$study)

[1] "ema"

[1] "1day"

[1] 0

[1] "v5"

[1] "nested"

[1] "main"

Function conflicts

In [ ]:
# source
devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/fun_ml.R?raw=true")

ℹ SHA-1 hash of file is "77e91675366f10788c6bcb59fa1cfc9ee0c75281"

Chunk Defaults

In [ ]:
knitr::opts_chunk$set(attr.output='style="max-height: 500px;"')

options(tibble.width = Inf)
options(tibble.print_max = Inf)

Packages for script

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Source support functions

In [ ]:
devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true")

ℹ SHA-1 hash of file is "a58e57da996d1b70bb9a5b58241325d6fd78890f"

Absolute paths

In [ ]:
path_fairema <- format_path(str_c("studydata/risk/data_processed/fairema"))

### Read in data

In [ ]:
df_sex <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "sex", "_", window, 
                                           "_", lead, "_", version, "_", cv, "_", model,
                                           ".csv")),
                      col_types = cols())
df_sex$attribute <- "sex"

df_race <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "race", "_", window, 
                                           "_", lead, "_", version, "_", cv, "_", model,
                                           ".csv")),
                    col_types = cols())
df_race$attribute <- "race"

df_income <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "income", "_", window, 
                                           "_", lead, "_", version, "_", cv, "_", model,
                                           ".csv")),
                      col_types = cols())
df_income$attribute <- "income"

df_age <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "age", "_", window, 
                                           "_", lead, "_", version, "_", cv, "_", model,
                                           ".csv")),
                   col_types = cols())
df_age$attribute <- "age"

df <- rbind(df_sex, df_race, df_income, df_age)

### Make Dataframes for plots

In [ ]:
mak_df_plot <- function(df){
  df |> 
    filter(group != "all", n_lapses != 0) |> 
    select(-n_lapses) |> 
    pivot_longer(-c("outer_split_num", "group", "attribute"),
               names_to = "metrics", values_to = "score") |>
    mutate(metrics = case_when(metrics == "balaccuracy" ~ "balanced accuracy",
                               metrics != "balaccuracy" ~ metrics),
           group = case_when(
             group == "Male" ~ "Privileged (Male)",
             group == "Female" ~ "Unprivileged (Female)",
             group == "white" ~ "Privileged (White)",
             group == "non_white" ~ "Unprivileged (Person of Color)",
             group == "above_poverty" ~ "Privileged (Above Poverty)",
             group == "below_poverty" ~ "Unprivileged (Below Poverty)",
             group == "younger" ~ "Privileged (Age 55 or older)",
             group == "older" ~ "Unprivileged (Below 55)"
           ),
           sig = case_when(
             attribute == "sex" & metrics == "auroc" ~ "**",
             attribute == "sex" & metrics == "prauc" ~ "",
             attribute == "sex" & metrics == "sensitivity" ~ "",
             attribute == "sex" & metrics == "specificity" ~ "*",
             attribute == "sex" & metrics == "ppv" ~ "",
             attribute == "sex" & metrics == "balanced accuracy" ~ "",
             attribute == "race" & metrics == "auroc" ~ "***",
             attribute == "race" & metrics == "prauc" ~ "***",
             attribute == "race" & metrics == "sensitivity" ~ "***",
             attribute == "race" & metrics == "specificity" ~ "*",
             attribute == "race" & metrics == "ppv" ~ "*",
             attribute == "race" & metrics == "balanced accuracy" ~ "***",
             attribute == "income" & metrics == "auroc" ~ "**",
             attribute == "income" & metrics == "prauc" ~ "*",
             attribute == "income" & metrics == "sensitivity" ~ "***",
             attribute == "income" & metrics == "specificity" ~ "",
             attribute == "income" & metrics == "ppv" ~ "",
             attribute == "income" & metrics == "balanced accuracy" ~ "***",
             attribute == "age" & metrics == "auroc" ~ "***",
             attribute == "age" & metrics == "prauc" ~ "",
             attribute == "age" & metrics == "sensitivity" ~ "**",
             attribute == "age" & metrics == "specificity" ~ "*",
             attribute == "age" & metrics == "ppv" ~ "*",
             attribute == "age" & metrics == "balanced accuracy" ~ "***"
           ),
           CI = case_when(
             attribute == "sex" & metrics == "auroc" ~ "CI[0.014, 0.074]",
             attribute == "sex" & metrics == "prauc" ~ "CI[-0.080, 0.106]",
             attribute == "sex" & metrics == "sensitivity" ~ "CI[-0.065, 0.106]",
             attribute == "sex" & metrics == "specificity" ~ "CI[0.011, 0.101]",
             attribute == "sex" & metrics == "ppv" ~ "CI[-0.071, 0.057]",
             attribute == "sex" & metrics == "balanced accuracy" ~ "CI[-0.001, 0.078]",
             attribute == "race" & metrics == "auroc" ~ "CI[0.127, 0.259]",
             attribute == "race" & metrics == "prauc" ~ "CI[0.121, 0.324]",
             attribute == "race" & metrics == "sensitivity" ~ "CI[0.132, 0.363]",
             attribute == "race" & metrics == "specificity" ~ "CI[0.0103, 0.152]",
             attribute == "race" & metrics == "ppv" ~ "CI[0.026, 0.182]",
             attribute == "race" & metrics == "balanced accuracy" ~ "CI[0.121, 0.206]",
             attribute == "income" & metrics == "auroc" ~ "CI[0.067, 0.196]",
             attribute == "income" & metrics == "prauc" ~ "CI[0.016, 0.227]",
             attribute == "income" & metrics == "sensitivity" ~ "CI[0.140, 0.344]",
             attribute == "income" & metrics == "specificity" ~ "CI[-0.103, -0.008]",
             attribute == "income" & metrics == "ppv" ~ "CI[-0.079, 0.089]",
             attribute == "income" & metrics == "balanced accuracy" ~ "CI[0.047, 0.139]",
             attribute == "age" & metrics == "auroc" ~ "CI[0.057, 0.138]",
             attribute == "age" & metrics == "prauc" ~ "CI[-0.017, 0.187]",
             attribute == "age" & metrics == "sensitivity" ~ "CI[0.063, 0.240]",
             attribute == "age" & metrics == "specificity" ~ "CI[0.006, 0.106]",
             attribute == "age" & metrics == "ppv" ~ "CI[0.011, 0.143]",
             attribute == "age" & metrics == "balanced accuracy" ~ "CI[0.065, 0.149]"
           )
           )|> 
    mutate(metrics = if_else(metrics == "prauc", "auPR", metrics)) |> 
    mutate(metrics = if_else(metrics == "auroc", "auROC", metrics))
}

In [ ]:
df_sex <- mak_df_plot(df_sex)
df_race <- mak_df_plot(df_race)
df_income <- mak_df_plot(df_income)
df_age <- mak_df_plot(df_age)
df <- mak_df_plot(df)

### Construct Plots

In [ ]:
sig_marks <- c("", "*", "**", "***", "")



r1 <-     df_race |> 
      filter(metrics %in% c("auROC", "auPR")) |> 
      ggplot(aes(x = "", y = score, fill = group)) +
      geom_boxplot(outlier.size = .2) +
      facet_wrap(~factor(metrics, levels = c("auROC", "auPR"))) +
      theme_classic() +
      theme(axis.title.y = element_blank(),
            axis.ticks.y = element_blank(),
            axis.line.y = element_blank(),
            axis.text.y = element_text(margin = margin(r = -12)),
            axis.line.x = element_blank(),
            axis.ticks.x = element_blank(),
            plot.margin = margin(1, 1, 1, 1, "pt"),
            legend.text = element_text(size=7),
            legend.title=element_blank()) +
      scale_fill_manual(values = c("#6CAA2F", "#C23E28")) +
      labs(x = NULL) +
      guides(fill=guide_legend(nrow=2, byrow=TRUE)) +
      geom_text(aes(label = sig, y = 0), color = "blue", size = 7)+ 
  theme(
    panel.background = element_rect(fill='transparent'), #transparent panel bg
    plot.background = element_rect(fill='transparent', color=NA), #transparent plot bg
    panel.grid.major = element_blank(), #remove major gridlines
    panel.grid.minor = element_blank(), #remove minor gridlines
    strip.background =element_rect(fill="transparent"),
    legend.background = element_rect(fill='transparent'), #transparent legend bg
    legend.box.background = element_rect(fill='transparent') #transparent legend panel
  )
    
r2 <-     df_race |> 
      filter(metrics %in% c("sensitivity", "specificity", "ppv")) |> 
      ggplot(aes(x = "", y = score, fill = group)) +
      geom_boxplot(outlier.size = .2) +
      facet_wrap(~factor(metrics, 
                         levels = c("specificity", "sensitivity", "ppv"))) +
      theme_classic() +
      theme(axis.title.y = element_blank(),
            axis.ticks.y = element_blank(),
            axis.line.y = element_blank(),
            axis.text.y = element_text(margin = margin(r = -12)),
            axis.line.x = element_blank(),
            axis.ticks.x = element_blank(),
            plot.margin = margin(1, 1, 1, 1, "pt"),
            legend.text = element_text(size=7),
            legend.title=element_blank()) + 
      scale_fill_manual(values = c("#6CAA2F", "#C23E28")) +
      labs(x = NULL) +
      guides(fill=guide_legend(nrow=2, byrow=TRUE)) +
      geom_text(aes(label = sig, y = 0), color = "blue", size = 7)+ 
  theme(
    panel.background = element_rect(fill='transparent'), #transparent panel bg
    plot.background = element_rect(fill='transparent', color=NA), #transparent plot bg
    panel.grid.major = element_blank(), #remove major gridlines
    panel.grid.minor = element_blank(), #remove minor gridlines
    strip.background =element_rect(fill="transparent"),
    legend.background = element_rect(fill='transparent'), #transparent legend bg
    legend.box.background = element_rect(fill='transparent') #transparent legend panel
  )

r <- ggarrange(r1, r2, ncol = 1, common.legend = TRUE, legend = "bottom")
r <- annotate_figure(r, top = text_grob("Race", face = "bold"))

In [ ]:
s1 <-     df_sex |> 
      filter(metrics %in% c("auROC", "auPR")) |> 
      ggplot(aes(x = "", y = score, fill = group)) +
      geom_boxplot(outlier.size = .2) +
      facet_wrap(~factor(metrics, levels = c("auROC", "auPR"))) +
      theme_classic() +
      theme(axis.title.y = element_blank(),
            axis.ticks.y = element_blank(),
            axis.line.y = element_blank(),
            axis.text.y = element_text(margin = margin(r = -12)),
            axis.line.x = element_blank(),
            axis.ticks.x = element_blank(),
            plot.margin = margin(1, 1, 1, 1, "pt"),
            legend.text = element_text(size=7),
            legend.title=element_blank()) +
      scale_fill_manual(values = c("#6CAA2F", "#C23E28")) +
      labs(x = NULL) +
      guides(fill=guide_legend(nrow=2, byrow=TRUE)) +
      geom_text(aes(label = sig, y = 0), color = "blue", size = 7)+
  theme(
    panel.background = element_rect(fill='transparent'), #transparent panel bg
    plot.background = element_rect(fill='transparent', color=NA), #transparent plot bg
    panel.grid.major = element_blank(), #remove major gridlines
    panel.grid.minor = element_blank(), #remove minor gridlines
    strip.background =element_rect(fill="transparent"),
    legend.background = element_rect(fill='transparent'), #transparent legend bg
    legend.box.background = element_rect(fill='transparent') #transparent legend panel
  )
    
s2 <-     df_sex |> 
      filter(metrics %in% c("sensitivity", "specificity", "ppv")) |> 
      ggplot(aes(x = "", y = score, fill = group)) +
      geom_boxplot(outlier.size = .2) +
      facet_wrap(~factor(metrics, 
                         levels = c("specificity", "sensitivity", "ppv"))) +
      theme_classic() +
      theme(axis.title.y = element_blank(),
            axis.ticks.y = element_blank(),
            axis.line.y = element_blank(),
            axis.text.y = element_text(margin = margin(r = -12)),
            axis.line.x = element_blank(),
            axis.ticks.x = element_blank(),
            plot.margin = margin(1, 1, 1, 1, "pt"),
            legend.text = element_text(size=7),
            legend.title=element_blank()) + 
      scale_fill_manual(values = c("#6CAA2F", "#C23E28")) +
      labs(x = NULL) +
      guides(fill=guide_legend(nrow=2, byrow=TRUE)) +
      geom_text(aes(label = sig, y = 0), color = "blue", size = 7)+ 
  theme(
    panel.background = element_rect(fill='transparent'), #transparent panel bg
    plot.background = element_rect(fill='transparent', color=NA), #transparent plot bg
    panel.grid.major = element_blank(), #remove major gridlines
    panel.grid.minor = element_blank(), #remove minor gridlines
    strip.background =element_rect(fill="transparent"),
    legend.background = element_rect(fill='transparent'), #transparent legend bg
    legend.box.background = element_rect(fill='transparent') #transparent legend panel
  )


s <- ggarrange(s1, s2, ncol = 1, common.legend = TRUE, legend = "bottom")
s <- annotate_figure(s, top = text_grob("Sex", face = "bold"))

In [ ]:
i1 <-     df_income |> 
      filter(metrics %in% c("auROC", "auPR")) |> 
      ggplot(aes(x = "", y = score, fill = group)) +
      geom_boxplot(outlier.size = .2) +
      facet_wrap(~factor(metrics, levels = c("auROC", "auPR"))) +
      theme_classic() +
      theme(axis.title.y = element_blank(),
            axis.ticks.y = element_blank(),
            axis.line.y = element_blank(),
            axis.text.y = element_text(margin = margin(r = -12)),
            axis.line.x = element_blank(),
            axis.ticks.x = element_blank(),
            plot.margin = margin(1, 1, 1, 1, "pt"),
            legend.text = element_text(size=7),
            legend.title=element_blank()) +
      scale_fill_manual(values = c("#6CAA2F", "#C23E28")) +
      labs(x = NULL) +
      guides(fill=guide_legend(nrow=2, byrow=TRUE)) +
      geom_text(aes(label = sig, y = 0), color = "blue", size = 7)+ theme(legend.position="none")+
  theme(
    panel.background = element_rect(fill='transparent'), #transparent panel bg
    plot.background = element_rect(fill='transparent', color=NA), #transparent plot bg
    panel.grid.major = element_blank(), #remove major gridlines
    panel.grid.minor = element_blank(), #remove minor gridlines
    strip.background =element_rect(fill="transparent"),
    legend.background = element_rect(fill='transparent'), #transparent legend bg
    legend.box.background = element_rect(fill='transparent') #transparent legend panel
  )
    
i2 <-     df_income |> 
      filter(metrics %in% c("sensitivity", "specificity", "ppv")) |> 
      ggplot(aes(x = "", y = score, fill = group)) +
      geom_boxplot(outlier.size = .2) +
      facet_wrap(~factor(metrics, 
                         levels = c("specificity", "sensitivity", "ppv"))) +
      theme_classic() +
      theme(axis.title.y = element_blank(),
            axis.ticks.y = element_blank(),
            axis.line.y = element_blank(),
            axis.text.y = element_text(margin = margin(r = -12)),
            axis.line.x = element_blank(),
            axis.ticks.x = element_blank(),
            plot.margin = margin(1, 1, 1, 1, "pt"),
            legend.text = element_text(size=7),
            legend.title=element_blank()) + 
      scale_fill_manual(values = c("#6CAA2F", "#C23E28")) +
      labs(x = NULL) +
      guides(fill=guide_legend(nrow=2, byrow=TRUE)) +
      geom_text(aes(label = sig, y = 0), color = "blue", size = 7)+ theme(legend.position="none")+
  theme(
    panel.background = element_rect(fill='transparent'), #transparent panel bg
    plot.background = element_rect(fill='transparent', color=NA), #transparent plot bg
    panel.grid.major = element_blank(), #remove major gridlines
    panel.grid.minor = element_blank(), #remove minor gridlines
    strip.background =element_rect(fill="transparent"),
    legend.background = element_rect(fill='transparent'), #transparent legend bg
    legend.box.background = element_rect(fill='transparent') #transparent legend panel
  )


i <- ggarrange(i1, i2, ncol = 1, common.legend = TRUE, legend = "bottom")

In [ ]:
a1 <-     df_age |> 
      filter(metrics %in% c("auROC", "auPR")) |> 
      ggplot(aes(x = "", y = score, fill = group)) +
      geom_boxplot(outlier.size = .2) +
      facet_wrap(~factor(metrics, levels = c("auROC", "auPR"))) +
      theme_classic() +
      theme(axis.title.y = element_blank(),
            axis.ticks.y = element_blank(),
            axis.line.y = element_blank(),
            axis.text.y = element_text(margin = margin(r = -12)),
            axis.line.x = element_blank(),
            axis.ticks.x = element_blank(),
            plot.margin = margin(1, 1, 1, 1, "pt"),
            legend.text = element_text(size=7),
            legend.title=element_blank()) +
      scale_fill_manual(values = c("#6CAA2F", "#C23E28")) +
      labs(x = NULL) +
      guides(fill=guide_legend(nrow=2, byrow=TRUE)) +
      geom_text(aes(label = sig, y = 0), color = "blue", size = 7)+ theme(legend.position="none")+
  theme(
    panel.background = element_rect(fill='transparent'), #transparent panel bg
    plot.background = element_rect(fill='transparent', color=NA), #transparent plot bg
    panel.grid.major = element_blank(), #remove major gridlines
    panel.grid.minor = element_blank(), #remove minor gridlines
    strip.background =element_rect(fill="transparent"),
    legend.background = element_rect(fill='transparent'), #transparent legend bg
    legend.box.background = element_rect(fill='transparent') #transparent legend panel
  )
    
a2 <-     df_age  |> 
      filter(metrics %in% c("sensitivity", "specificity", "ppv")) |> 
      ggplot(aes(x = "", y = score, fill = group)) +
      geom_boxplot(outlier.size = .2) +
      facet_wrap(~factor(metrics, 
                         levels = c("specificity", "sensitivity", "ppv"))) +
      theme_classic() +
      theme(axis.title.y = element_blank(),
            axis.ticks.y = element_blank(),
            axis.line.y = element_blank(),
            axis.text.y = element_text(margin = margin(r = -12)),
            axis.line.x = element_blank(),
            axis.ticks.x = element_blank(),
            plot.margin = margin(1, 1, 1, 1, "pt"),
            legend.text = element_text(size=7),
            legend.title=element_blank()) + 
      scale_fill_manual(values = c("#6CAA2F", "#C23E28")) +
      labs(x = NULL) +
      guides(fill=guide_legend(nrow=2, byrow=TRUE)) +
      geom_text(aes(label = sig, y = 0), color = "blue", size = 7)+ theme(legend.position="none")+
  theme(
    panel.background = element_rect(fill='transparent'), #transparent panel bg
    plot.background = element_rect(fill='transparent', color=NA), #transparent plot bg
    panel.grid.major = element_blank(), #remove major gridlines
    panel.grid.minor = element_blank(), #remove minor gridlines
    strip.background =element_rect(fill="transparent"),
    legend.background = element_rect(fill='transparent'), #transparent legend bg
    legend.box.background = element_rect(fill='transparent') #transparent legend panel
  )


a <- ggarrange(a1, a2, ncol = 1, common.legend = TRUE, legend = "bottom")
a <- annotate_figure(a, top = text_grob("Age", face = "bold"))

In [ ]:
sr <- ggarrange(
  r,
  s
)

ia <- ggarrange(
  i,
  a
)

In [ ]:
#set printing defaults for ggsave
gg_width <-  4


#printing

ggsave("_presentations/poster1/images/barplot_race1.png", r1,  bg="transparent", width = gg_width, height = gg_width)
ggsave("_presentations/poster1/images/barplot_race2.png", r2, bg="transparent", width = gg_width, height = gg_width)
ggsave("_presentations/poster1/images/barplot_sex1.png", s1,  bg="transparent", width = gg_width, height = gg_width)
ggsave("_presentations/poster1/images/barplot_sex2.png", s2, bg="transparent", width = gg_width, height = gg_width)
ggsave("_presentations/poster1/images/barplot_income1.png", i1,  bg="transparent", width = gg_width, height = gg_width)
ggsave("_presentations/poster1/images/barplot_income2.png", i2, bg="transparent", width = gg_width, height = gg_width)